In [302]:
# Análisis de sentimientos en reddit

In [303]:
# Importación de librerías

In [304]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import praw

In [305]:
# Creación de Aplicación de desarrollador en reddit

In [306]:
user_agent = "Scraper 1"
reddit = praw.Reddit (
    client_id = "qMrF90radmJgqcs1Vy-Hgg",
    client_secret = "LVzl0UiY5-zFYQwv-Segm40aHsxTCw",
    user_agent = user_agent
)

In [307]:
# Extraemos y mostramos el primer comentario extraido

In [308]:
headlines = set()
for Submission in reddit.subreddit('politics').hot(limit=None):
    print(Submission.title)
    print(Submission.id)
    print(Submission.author)
    print(Submission.created_utc)
    print(Submission.score)
    print(Submission.upvote_ratio)
    print(Submission.url)
    break
    headlines.add(Submission.title)
    print(len(headlines))

The "What happened in your state last week" Megathread, Week 7 & 8
1b0mh78
optimalg
1708966806.0
51
0.9
https://www.reddit.com/r/politics/comments/1b0mh78/the_what_happened_in_your_state_last_week/


In [309]:
# Extraemos comentarios y almacenamos en headlines. Se muestra la cantidad de comentarios extraidos

In [310]:
headlines = set()
for Submission in reddit.subreddit('politics').hot(limit=None):
    headlines.add(Submission.title)
print(len(headlines))


666


In [311]:
# Se crea un dataframe donde se almacenan los comentarios y se visualizan los 5 primeros

In [312]:
df = pd.DataFrame(headlines)
df.head()

0
0       Trump’s Next Legal Move: Personal Bankruptcy
1  Former Australian PM Malcolm Turnbull says Don...
2  Former U.S. spies warned in 2020 that the Hunt...
3  Trump's lawyers seek to suspend $83M defamatio...
4  NYC’s non-citizen voting law ruled unconstitut...

In [313]:
# Se convierte el dataframe a formato csv

In [314]:
df.to_csv('redditheadlines.csv', header = False, encoding = 'utf-8', index = False)

In [315]:
# Se inicia el análisis con Vader

In [316]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pprint import pprint

sia = SentimentIntensityAnalyzer()

results = []

# Contador para limitar el número de noticias a procesar
count = 0

for line in headlines:
    # Procesar solo las primeras 3 noticias
    if count < 600:
        pol_score = sia.polarity_scores(line)
        pol_score['headlines'] = line
        results.append(pol_score)
        count += 1
    else:
        break  # Salir del bucle una vez que se procesen las primeras 3 noticias


In [317]:
# Se visualian los 5 primeros comentarios con los resultados

In [318]:
df = pd.DataFrame.from_records(results)
df.head()

neg    neu    pos  compound  \
0  0.000  0.769  0.231    0.1280   
1  0.147  0.649  0.203    0.0772   
2  0.219  0.658  0.123   -0.2960   
3  0.126  0.604  0.269    0.3818   
4  0.000  1.000  0.000    0.0000   

                                           headlines  
0       Trump’s Next Legal Move: Personal Bankruptcy  
1  Former Australian PM Malcolm Turnbull says Don...  
2  Former U.S. spies warned in 2020 that the Hunt...  
3  Trump's lawyers seek to suspend $83M defamatio...  
4  NYC’s non-citizen voting law ruled unconstitut...

In [319]:
# Se categorizan los comentarios por su tendencia

# label = 1 es positivo
# label = -1 es negativo

df['label'] = 0
df.loc[df['compound']>0.2, 'label'] = 1
df.loc[df['compound']<-0.2, 'label'] = -1
df.head()

neg    neu    pos  compound  \
0  0.000  0.769  0.231    0.1280   
1  0.147  0.649  0.203    0.0772   
2  0.219  0.658  0.123   -0.2960   
3  0.126  0.604  0.269    0.3818   
4  0.000  1.000  0.000    0.0000   

                                           headlines  label  
0       Trump’s Next Legal Move: Personal Bankruptcy      0  
1  Former Australian PM Malcolm Turnbull says Don...      0  
2  Former U.S. spies warned in 2020 that the Hunt...     -1  
3  Trump's lawyers seek to suspend $83M defamatio...      1  
4  NYC’s non-citizen voting law ruled unconstitut...      0

In [320]:
# -1 representa negativo, 0 neutral y 1 positivo

In [321]:
df.label.value_counts()

label
-1    246
 0    220
 1    134
Name: count, dtype: int64

In [322]:
# Se normalizan los datos

In [323]:
df.label.value_counts(normalize = True)*100

label
-1    41.000000
 0    36.666667
 1    22.333333
Name: proportion, dtype: float64

In [324]:
# Se muestran ejemplos comentarios negativos y positivos

In [325]:
print("Positive headlines:\n")
pprint(list(df[df['label']==1].headlines)[:5],width=200)

print("Negative headlines:\n")
pprint(list(df[df['label']==-1].headlines)[:5],width=200)


Positive headlines:

["Trump's lawyers seek to suspend $83M defamation verdict, citing 'strong probability' it won't stand",
 "Trump Says He's Too Rich To Post Bond In Carroll Defamation Case",
 'Jamaal Bowman, losing favor among Israel supporters, teams up with Rashida Tlaib',
 'Republicans who say they support IVF backed a bill protecting life ‘at conception’',
 'CPAC security slammed after forcing special needs child to move seats']
Negative headlines:

['Former U.S. spies warned in 2020 that the Hunter Biden scandal had Russian fingerprints. They feel vindicated now.',
 'West Virginia Republicans want to prevent suicidal transgender youth from receiving care. GOP lawmakers in West Virginia are pushing to overturn an exception that allows transgender youth at risk '
 'of self harm or suicide to receive gender-affirming care.',
 "Alabama’s IVF Ruling Is a Dire Warning to Other States: The state's embrace of fetal personhood has had terrible, if unintended, consequences that will kill

In [326]:
# Se importan librerías de evaluación

In [327]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
import itertools

In [328]:
# Se leen los datos en un nuevo datframe

In [329]:
data = pd.read_csv('reddit_headlines_label.csv')

In [330]:
data.head(10)

headlines  label
0       Trump’s Next Legal Move: Personal Bankruptcy      0
1  Former Australian PM Malcolm Turnbull says Don...      0
2  Former U.S. spies warned in 2020 that the Hunt...     -1
3  Trump's lawyers seek to suspend $83M defamatio...      1
4  NYC’s non-citizen voting law ruled unconstitut...      0
5  One statistic captures Washington’s indefensib...      0
6  Trump Says He's Too Rich To Post Bond In Carro...      1
7  West Virginia Republicans want to prevent suic...     -1
8  Alabama’s IVF Ruling Is a Dire Warning to Othe...     -1
9  Trump pushes for delay in fraud trial ruling. ...     -1

In [331]:
data["label"].value_counts()

label
-1    247
 0    219
 1    134
Name: count, dtype: int64

In [332]:
# Se dividen los datos en conjunto de entrenamiento y prueba

In [333]:
X = data["headlines"]
y = data["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [334]:
len(y_train)

300

In [335]:
# Se crea una matriz de tokens de palabras y se evalua con el clasificador Naive Bayes

In [336]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()), ])

In [337]:
text_clf = text_clf.fit(X_train, y_train)
predicted1 = text_clf.predict(X_test)

In [338]:
metrics.accuracy_score(y_test, predicted1)

0.4666666666666667

In [339]:
print(metrics.classification_report(y_test, predicted1, target_names=['Negativo', 'Neutral', 'Positivo']))


              precision    recall  f1-score   support

    Negativo       0.50      0.63      0.56       130
     Neutral       0.40      0.50      0.44       104
    Positivo       0.86      0.09      0.16        66

    accuracy                           0.47       300
   macro avg       0.59      0.41      0.39       300
weighted avg       0.55      0.47      0.43       300



In [340]:
# Se evalua con el método de Regresión logística

In [341]:
X_train, X_test, y_train, y_test = train_test_split(data['headlines'], data['label'], test_size=0.3, random_state=42)

In [342]:
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

In [343]:
tfidf_vect = TfidfVectorizer(stop_words = 'english', max_df = 0.25)
tfidf_train = tfidf_vect.fit_transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)

In [344]:
logmodel = LogisticRegression()
logmodel.fit(tfidf_train,y_train)

LogisticRegression()

In [345]:
predictions = logmodel.predict(tfidf_test)
predictions

array([-1, -1, -1, -1, -1, -1,  0, -1,  0,  0,  0, -1,  0,  0,  0,  0,  0,
        0, -1, -1, -1, -1,  0,  0, -1, -1,  0,  0, -1,  0,  0, -1,  0, -1,
        0, -1,  0, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1,  0,  0, -1, -1,
        0, -1,  1,  0, -1,  0,  0, -1,  0, -1, -1,  0,  0, -1,  0, -1, -1,
       -1,  1,  0, -1,  0, -1,  0,  0, -1, -1,  0,  1, -1, -1, -1,  0,  0,
       -1,  0,  0, -1,  1,  0, -1, -1,  0,  0,  0,  0,  0, -1,  0,  0, -1,
       -1,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0, -1, -1, -1,  0, -1,  0,
       -1,  0, -1,  0,  0,  0, -1, -1,  0, -1, -1,  0, -1,  0, -1,  0, -1,
       -1,  1, -1,  0,  0, -1, -1, -1,  0,  0,  0, -1, -1, -1,  0, -1,  0,
       -1, -1,  0, -1, -1,  0,  1, -1, -1,  0, -1, -1, -1,  0, -1, -1, -1,
        0, -1, -1,  0, -1,  0,  0,  0, -1, -1], dtype=int64)

In [346]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

          -1       0.65      0.71      0.67        85
           0       0.40      0.57      0.47        56
           1       0.67      0.10      0.18        39

    accuracy                           0.53       180
   macro avg       0.57      0.46      0.44       180
weighted avg       0.57      0.53      0.50       180

